In [1]:
import sqlite3
import pandas as pd

In [2]:
conn = sqlite3.connect('PBS.db')
builder_proposer_tx = pd.read_sql_query(
    "SELECT * FROM builder_proposer_tx", conn)
relay_data = pd.read_sql_query("SELECT * FROM relay_data", conn)
df = pd.merge(builder_proposer_tx, relay_data, on='block_hash')
df = df[['relay', 'block_number_x', 'builder_pubkey', 'to_address']]
df = df.rename(columns={'block_number_x': 'block_number', 'to_address': 'proposer_address'})

In [11]:
df

,relay,block_number,builder_pubkey,proposer_address
0,agnostic Gnosis,18951601,0x978a35c39c41aadbe35ea29712bccffb117cc6ebcad4...,0xb3d9cf8e163bbc840195a97e81f8a34e295b8f39
1,ultrasound,18951601,0x978a35c39c41aadbe35ea29712bccffb117cc6ebcad4...,0xb3d9cf8e163bbc840195a97e81f8a34e295b8f39
2,flashbots,18951918,0xae7f0dd473d986b277539cca610aba02c437082ac504...,0x0000000000450702bc4f750fd1e7ecad7054c4f1
3,agnostic Gnosis,18951918,0xae7f0dd473d986b277539cca610aba02c437082ac504...,0x0000000000450702bc4f750fd1e7ecad7054c4f1
4,ultrasound,18951918,0xae7f0dd473d986b277539cca610aba02c437082ac504...,0x0000000000450702bc4f750fd1e7ecad7054c4f1
...,...,...,...,...
196814,agnostic Gnosis,18911286,0x83d3495a2951065cf19c4d282afca0a635a39f6504bd...,0xd6e4aa932147a3fe5311da1b67d9e73da06f9cef
196815,aestus,19089909,0xb0a3a5e8e0330698176d35fda999f522d99222597559...,0xd6e4aa932147a3fe5311da1b67d9e73da06f9cef
196816,ultrasound,19091524,0xaec4ec48c2ec03c418c599622980184e926f0de3c9ce...,0xd6e4aa932147a3fe5311da1b67d9e73da06f9cef
196817,flashbots,19093173,0x978a35c39c41aadbe35ea29712bccffb117cc6ebcad4...,0xd6e4aa932147a3fe5311da1b67d9e73da06f9cef


In [4]:
def get_relay_gini(df):
    relay_df = df.groupby('relay')['block_number'].nunique().reset_index()
    relay_df.columns = ['relay', 'cumulative_block_number']

    relays = relay_df['relay'].unique()
    relay_blocks = relay_df['cumulative_block_number']

    numerator = 0
    denominator = 2 * len(relays) * relay_blocks.sum()

    for i in range(len(relays)):
        for j in range(i+1, len(relays)):
            numerator += abs(relay_blocks.iloc[i] - relay_blocks.iloc[j])

    return numerator / denominator

In [5]:
def get_builder_gini(df):
    builder_df = df.groupby('builder_pubkey')['block_number'].nunique().reset_index()
    builder_df.columns = ['builder_pubkey', 'cumulative_block_number']

    builders = builder_df['builder_pubkey'].unique()
    builder_blocks = builder_df['cumulative_block_number']

    numerator = 0
    denominator = 2 * len(builders) * builder_blocks.sum()

    for i in range(len(builders)):
        for j in range(i+1, len(builders)):
            numerator += abs(builder_blocks.iloc[i] - builder_blocks.iloc[j])

    return numerator / denominator

In [6]:
def get_proposer_gini(df):
    proposer_df = df.groupby('proposer_address')[
        'block_number'].nunique().reset_index()
    proposer_df.columns = ['proposer_address', 'cumulative_block_number']

    proposers = proposer_df['proposer_address'].unique()
    proposer_blocks = proposer_df['cumulative_block_number']

    numerator = 0
    denominator = 2 * len(proposers) * proposer_blocks.sum()

    for i in range(len(proposers)):
        for j in range(i+1, len(proposers)):
            numerator += abs(proposer_blocks.iloc[i] - proposer_blocks.iloc[j])

    return numerator / denominator

In [7]:
get_relay_gini(df)

0.2519281675041536

In [8]:
get_builder_gini(df)

0.37463875373346783

In [9]:
get_proposer_gini(df)

0.4628380540053418